In [1]:
import torch
import torchvision
from torchvision import transforms,utils
train_dataset=torchvision.datasets.ImageFolder("/home/user01/envoriments/data/train/coat_length_labels/",
                                         transform=transforms.Compose([
                                            transforms.Scale(224),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor()
                                         ]))
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=6)

validate_dataset=torchvision.datasets.ImageFolder("/home/user01/envoriments/data/validate/coat_length_labels/",
                                         transform=transforms.Compose([
                                            transforms.Scale(224),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor()
                                         ]))
validate_dataloader=torch.utils.data.DataLoader(validate_dataset,batch_size=10,shuffle=True,num_workers=6)

dset_loaders={'train':train_dataloader,'validate':validate_dataloader}
dset_sizes={'train':len(train_dataset),'validate':len(validate_dataset)}
use_gpu=torch.cuda.is_available()
print("process end")

process end


In [2]:
from torch.autograd import Variable
import time
def train_model(model,criterion,optimizer,lr_scheduler,num_epochs=30):
    since=time.time()
    best_model=model
    best_acc=0.0
    for epoch in range(num_epochs):
        print('epoch:{}/{}'.format(epoch,num_epochs-1))
        print('_'*10)
        for phase in ['train','validate']:
            if phase=='train':
                optimizer=lr_scheduler(optimizer,epoch)
                model.train(True)
            else:
                model.train(False)
            running_loss=0.0
            running_corrects=0
            for data in dset_loaders[phase]:
                inputs,labels=data
                if use_gpu:
                    inputs,labels=Variable(inputs.cuda()),Variable(labels.cuda())
                else:
                    inputs,labels=Variable(inputs),Variable(labels)
                optimizer.zero_grad()
                outputs=model(inputs)
                _,preds=torch.max(outputs.data,1)#返回第一维的值和索引
                loss=criterion(outputs,labels)
                if phase=='train':
                    loss.backward()
                    optimizer.step()
                running_loss+=loss.data[0]#?
                running_corrects+=torch.sum(preds==labels.data)
            epoch_loss=running_loss/dset_sizes[phase]
            epoch_acc=running_corrects/dset_sizes[phase]
            
            print('{} loss:{:.4f} acc:{:.4f}'.format(phase,epoch_loss,epoch_acc))
            if phase =="validate"and epoch_acc>best_acc:
                best_acc=epoch_acc
                best_model=copy.deepcopy(model)#?
    time_elapsed=time.time()-since
    print('花费时间为{:.0f}分：{:.0f}秒'.format(time_elapsed//60,time_elapsed%60))
    print("最好的准确率为：{:.4f}".format(best_acc))
    return best_model
        
print("process end")

process end


In [3]:
def exp_lr_scheduler(optimizer,epoch,init_lr=0.001,lr_decay_epoch=5):
    lr=init_lr*(0.1**(epoch//lr_decay_epoch))
    if epoch%lr_decay_epoch==0:
        print("此时学习率为{}".format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr']=lr
    return optimizer 
print("process end")

process end


In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim



import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image, ImageFilter
import torch

from torch.autograd import Variable

import torchvision
from torchvision import datasets,transforms
import time
import copy
#model_dense=models.inception_v3(pretrained=True)
model_dense=models.densenet169(pretrained=True)
model_dense.classifier=torch.nn.Linear(1664,8)

if use_gpu:
    model_dense=model_dense.cuda()
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.SGD(model_dense.parameters(),lr=0.001,momentum=0.9)

model=train_model(model_dense,criterion,optimizer,exp_lr_scheduler,num_epochs=22)
torch.save(model.state_dict(),'densenet_fashionai_coat_en.pkl')
print("process end")